In [6]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [21]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
from openai import OpenAI

openai_client = OpenAI()

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [9]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
rag('how do I run kafka?')

'To run Kafka with Java, you need to execute the following command in your project directory:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you\'re using Python and encounter the "Module \'kafka\' not found" error when trying to run `producer.py`, you should create a virtual environment, activate it, and install the required packages as per `requirements.txt`:\n\n1. Create a virtual environment (run only once):\n   ```sh\n   python -m venv env\n   ```\n\n2. Activate the virtual environment:\n   - On MacOS/Linux:\n     ```sh\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```sh\n     env\\Scripts\\activate\n     ```\n\n3. Install the necessary packages:\n   ```sh\n   pip install -r ../requirements.txt\n   ```\n\nEnsure your Docker images are running if required. To deactivate the virtual environment when done, use:\n```sh\ndeactivate\n```'

In [18]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are also eligible to submit the homework assignments. However, make sure to pay attention to the deadlines for turning in the final projects to avoid leaving everything until the last minute.'

### RAG with vector search

In [7]:
from qdrant_client import QdrantClient, models

In [10]:
qd_client = QdrantClient("http://localhost:6333")

In [15]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [17]:
qd_client.delete_collection(collection_name=collection_name)

True

In [18]:
collection_name = "zoomcamp-faq"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [19]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
        )
    points.append(point)

In [23]:
points[4]

PointStruct(id=4, vector=Document(text='Course - What can I do before the course starts?You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.', 'section': 'General course-related questions', 'question': 'Course - What can I do before the course starts?', 'course': 'data-engineering-zoomcamp'})

In [24]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [25]:
question = "I just discovered the course. Can I still join?"

In [26]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [27]:
vector_search("How do I run Docker?")

vector_search is used


[{'text': 'First off, make sure you\'re running the latest version of Docker for Windows, which you can download from here. Sometimes using the menu to "Upgrade" doesn\'t work (which is another clear indicator for you to uninstall, and reinstall with the latest version)\nIf docker is stuck on starting, first try to switch containers by right clicking the docker symbol from the running programs and switch the containers from windows to linux or vice versa\n[Windows 10 / 11 Pro Edition] The Pro Edition of Windows can run Docker either by using Hyper-V or WSL2 as its backend (Docker Engine)\nIn order to use Hyper-V as its back-end, you MUST have it enabled first, which you can do by following the tutorial: Enable Hyper-V Option on Windows 10 / 11\nIf you opt-in for WSL2, you can follow the same steps as detailed in the tutorial here',
  'section': 'Module 1: Docker and Terraform',
  'question': "Docker - Docker won't start or is stuck in settings (Windows 10 / 11)",
  'course': 'data-engi

In [28]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [31]:
print(rag("How do I run Kafka?"))

vector_search is used
To run Kafka, you should follow these steps:

1. **Run Producer/Consumer/KStreams in Terminal**:
   Navigate to the project directory and use the command:
   ```
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```
   Make sure to replace `<jar_name>` with the actual name of the jar file you are working with.

2. **Check Kafka Broker with Docker**:
   If you encounter the error `kafka.errors.NoBrokersAvailable`, it is likely that your Kafka broker Docker container is not running. To resolve this:
   - Use `docker ps` to confirm the status of your containers.
   - Navigate to the docker compose YAML file folder and run `docker compose up -d` to start all the instances.

3. **Updating Scripts**:
   Ensure that in the scripts located in `src/main/java/org/example/` (e.g., `JsonConsumer.java`, `JsonProducer.java`), the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` points to the correct server URL. Also, make sure that th